LabelEncoder used for categorical values.
SimpleImputer used for missing values.

Categorical values that are missing can not be used in LabelEncoder

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder

pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [2]:
data = pd.read_csv("H:\\Hobby\\ML Dataset\\house_price\\train.csv")
test = pd.read_csv("H:\\Hobby\\ML Dataset\\house_price\\test.csv")

y = data.SalePrice
data=data.drop(['SalePrice'],axis=1)

In [74]:
cols = (data.dtypes=='object')
cols = list(cols[cols].index)


bad_data = [col for col in cols if data[col].isnull().any()]
bad_test = [col for col in cols if test[col].isnull().any()]

bad = set(bad_data) | set(bad_test)
#Alley

data['BsmtCond'] = data['BsmtCond'].fillna('TA')
data['BsmtExposure'] = data['BsmtExposure'].fillna('No')

test['Electrical'].describe()
# data = data.drop(bad, axis=1)
# test = test.drop(bad, axis=1)

# encoder = LabelEncoder()
# for col in cols:
#     if col not in bad:
#         data[col] = encoder.fit_transform(data[col])
#         test[col] = encoder.transform(test[col])

count      1459
unique        4
top       SBrkr
freq       1337
Name: Electrical, dtype: object

In [53]:
data['BsmtFinType1'].value_counts()

Unf    430
GLQ    418
ALQ    220
BLQ    148
Rec    133
LwQ     74
Name: BsmtFinType1, dtype: int64

In [60]:
BsmtFinType_mapping = {'Unf':1, 'GLQ':2, 'ALQ':3, 'BLQ':4, 'Rec':5, 'LwQ':6}
data['BsmtFinType1'] = data['BsmtFinType1'].map(BsmtFinType_mapping)
test['BsmtFinType1'] = test['BsmtFinType1'].map(BsmtFinType_mapping)
data['BsmtFinType2'] = data['BsmtFinType2'].map(BsmtFinType_mapping)
test['BsmtFinType2'] = test['BsmtFinType2'].map(BsmtFinType_mapping)

In [69]:
BsmtQual_mapping = {'TA':1, 'Gd':2, 'Ex':3, 'Fa':4}
data['BsmtQual'] = data['BsmtQual'].map(BsmtQual_mapping)
test['BsmtQual'] = test['BsmtQual'].map(BsmtQual_mapping)

In [143]:
imputer = SimpleImputer()

temp = test.copy()
data = pd.DataFrame(imputer.fit_transform(data))
test = pd.DataFrame(imputer.transform(test))

test.columns = temp.columns
test['Id']=test['Id'].astype('int')

X=data

In [144]:
xTrain, xVal, yTrain, yVal = train_test_split(X, y)

In [145]:
data = []

for x in range(70,120,5):
    model = RandomForestRegressor(random_state=0, n_estimators=x)
    model.fit(xTrain,yTrain)
    preds = model.predict(xVal)
    print(mean_absolute_error(yVal, preds), x)

19867.015381604695 70
19892.252420091325 75
19792.713184931505 80
19806.529089443997 85
19763.309619482498 90
19826.82281182408 95
19778.752246575343 100
19732.962504892366 105
19747.445230386053 110
19700.42592019059 115


In [146]:
final_model = RandomForestRegressor(n_estimators=136, random_state=0)
final_model.fit(X,y)
y_pred = final_model.predict(test)

output = pd.DataFrame({'Id': test.Id,
                     'SalePrice': y_pred})

output.to_csv('3rdSub.csv', index=False)